In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import fasttext
import fasttext.util
from tqdm import tqdm
tqdm.pandas()

In [3]:
ft = fasttext.load_model('../../clustering_func_div/cc.en.300.bin')
# get dimension
ft.get_dimension()

300

In [4]:
data_train = pd.read_csv("../data/6lakh_pipeline_company_companies_data_2021-07-21.csv")

/home/slintel/bert/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data_train.columns

Index(['revenue', 'rank', 'stock_exchange', 'open_requisitions_range',
       'stock_price', 'stock_symbol', '_id', 'company_city', 'company_country',
       'company_state', 'company_crunchbase_page', 'company_description',
       'company_facebook_page', 'company_funding', 'company_industry',
       'company_latest_funding_date', 'company_linkedin_page', 'company_name',
       'company_phone_number', 'company_products_services',
       'company_profile_image_url', 'company_sector', 'company_size',
       'company_twitter_page', 'company_type', 'company_website', 'ebitda',
       'fiscal_year', 'homepage_url', 'job_count', 'tags', 'lead_count',
       'updated_at', 'permalink', 'company_founded_at', 'geo_point',
       'company_continent', 'postal_code', 'company_address', 'sic_code',
       'naics_code', 'created_at', 'cb_id', 'revenue_range', 'flag',
       'nubela_id', 'linkedin_internal_id', 'tagline', 'linkedin_search_id',
       'linkedin_employee_count', 'linkedin_follower_coun

In [6]:
# from gensim.models.fasttext import FastText
# ft_model = FastText.load_fasttext_format('cc.en.300.bin', encoding='utf8')
# print(ft_model.wv['artificial'])

In [7]:
data_train = data_train[["_id", "company_products_services"]]

In [8]:
# # df = pd.read_excel("100K_designations.xlsx")
# # reading data
# # data_train = pd.read_csv("../data/6lakh_pipeline_company_companies_data_2021-07-21.csv")
# # data=pd.read_csv("../data/50k_data_crunchbase.csv")
# X_train=pd.read_csv('../scripts/X_train_raw.csv', lineterminator='\n')
# # X_test=pd.read_csv('../scripts/X_test_raw.csv')
# y_train=pd.read_csv('../scripts/y_train_raw.csv', lineterminator='\n')
# y_test=pd.read_csv('../scripts/y_test_raw.csv', lineterminator='\n')


# X_test = pd.read_csv('../scripts/X_test_raw.csv',
#                  lineterminator='\n')

In [9]:
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [10]:
# X_train.columns

In [11]:
# X_train = X_train[["_id","company_products_services"]]

In [12]:
train_cps = data_train.dropna(how = "any")
train_cps.reset_index(inplace = True)

In [13]:
print(train_cps.head())
print(len(train_cps))

   index                       _id  \
0      0  5b073192599b311ccc11b101   
1      1  5b0737d1599b311ccc11c3f4   
2      2  5b073905599b311ccc11c792   
3      5  5b07552e599b311ccc121919   
4      6  5b0755f4599b311ccc121b43   

                           company_products_services  
0  [Personalization, Sales Development, Sales Com...  
1              [e-learning Herramienta colaborativa]  
2  [Power electronic systems, Advanced control ha...  
3     [Education, Learning, Tutoring, Homework Help]  
4                  [fine chemicals chemical sharing]  
382108


In [14]:
train_cps.drop(columns = ["index"], axis = 1, inplace = True)
train_cps.head()

/home/slintel/bert/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,_id,company_products_services
0,5b073192599b311ccc11b101,"[Personalization, Sales Development, Sales Com..."
1,5b0737d1599b311ccc11c3f4,[e-learning Herramienta colaborativa]
2,5b073905599b311ccc11c792,"[Power electronic systems, Advanced control ha..."
3,5b07552e599b311ccc121919,"[Education, Learning, Tutoring, Homework Help]"
4,5b0755f4599b311ccc121b43,[fine chemicals chemical sharing]


In [15]:
def get_embeddings(list_name, ft):
    
    for keyword in list_name:
        avg_embedding = []
        f = ft.get_word_vector(keyword)
#         print("shape of word vector for single word", f.shape)
        avg_embedding.append(f)
    final_avg_vector = np.mean(avg_embedding, axis = 0) 
    
    return final_avg_vector

In [16]:
def convert_format(n):
    # strip brackets
    n_split_clean = n.strip('[]')
    # split by comma
    n_split = n_split_clean.split(",")
    #strip whitespaces of start and end
    n_split_list =[]
    for i in n_split:
        n_split_final = i.strip(" ")
        n_split_list.append(n_split_final)
    n_new = n_split_list
        
    return n_new

In [17]:
# data['company_products_services_embeddings'] = data.apply(lambda x: get_embeddings(x["company_products_services"], ft), axis = 1)
train_cps["company_products_services"] = train_cps.progress_apply(lambda x: convert_format(x["company_products_services"]), axis = 1)

100%|██████████| 382108/382108 [00:07<00:00, 50377.49it/s]
/home/slintel/bert/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
train_cps["company_products_services"][0]

['Personalization', 'Sales Development', 'Sales Communication', 'Workflow']

In [19]:
# for keyword in list_name:
#     avg_embedding = []
#     f = ft.get_word_vector(keyword)
#     print("shape of word vector for single word", f.shape())
#     avg_embedding.append(f)
# final_avg_vector = np.mean(avg_embedding, axis = 0) 
#     return final_avg_vector
    
    
    

In [20]:
train_cps['CPS_embeddings'] = train_cps.progress_apply(lambda x: get_embeddings(x["company_products_services"], ft), axis = 1)

100%|██████████| 382108/382108 [01:07<00:00, 5680.35it/s]
/home/slintel/bert/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
train_cps["CPS_embeddings"][0]

array([ 5.81681840e-02,  4.32600155e-02,  4.78967018e-02, -6.37340769e-02,
        3.28424722e-02,  5.97780459e-02, -5.91425449e-02,  4.65427265e-02,
        1.95542648e-02,  9.10749938e-03, -8.88578147e-02,  3.63049507e-02,
       -5.63906021e-02,  2.52024811e-02, -1.25449123e-02, -2.75662635e-02,
        9.62147042e-02,  3.05396281e-02,  4.69662063e-02, -9.85592511e-03,
        4.82240394e-02, -6.49489388e-02, -2.58193118e-03,  3.01636080e-03,
       -2.29687262e-02, -7.12520033e-02, -3.94501165e-02,  1.20565603e-02,
        6.86522648e-02,  1.58117101e-01, -7.44756602e-04,  2.76350174e-02,
       -8.53598863e-03, -1.48489792e-02, -5.16629778e-02,  3.43001098e-03,
       -2.72104200e-02, -5.22704646e-02, -9.04752873e-03,  1.13058267e-02,
        4.79351915e-02, -4.31932025e-02, -4.85462463e-03,  3.92522998e-02,
       -4.62081395e-02, -9.25210188e-04, -2.49267332e-02, -2.56999228e-02,
        8.38824809e-02,  2.77770348e-02, -4.60156016e-02,  5.18320948e-02,
        6.70302585e-02, -

In [ ]:
# train_cps.to_csv("../data/cps_embeddings.csv", index = False)

In [22]:
train_cps.head()

,_id,company_products_services,CPS_embeddings
0,5b073192599b311ccc11b101,"[Personalization, Sales Development, Sales Com...","[0.058168184, 0.043260016, 0.0478967, -0.06373..."
1,5b0737d1599b311ccc11c3f4,[e-learning Herramienta colaborativa],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,5b073905599b311ccc11c792,"[Power electronic systems, Advanced control ha...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,5b07552e599b311ccc121919,"[Education, Learning, Tutoring, Homework Help]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,5b0755f4599b311ccc121b43,[fine chemicals chemical sharing],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
feat_cols = ['feature'+str(i) for i in tqdm(range(len(train_cps["CPS_embeddings"][0])))]

100%|██████████| 300/300 [00:00<00:00, 570912.52it/s]


In [24]:
# split array entries into individual columns
train_cps[feat_cols] = pd.DataFrame(train_cps.CPS_embeddings.tolist(), index= train_cps.index)

/home/slintel/bert/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [25]:
from sklearn.decomposition import PCA
# pca_breast = PCA(n_components=2)
# principalComponents_breast = pca_breast.fit_transform(x)

In [26]:
pca_breast = PCA(n_components=2)

In [27]:
# from sklearn.preprocessing import StandardScaler
# x = df_ip.loc[:, feat_cols].values
# x = StandardScaler().fit_transform(x) # normalizing the features

In [28]:
x = train_cps[feat_cols]

In [29]:
np.mean(x),np.std(x)


(feature0      0.001497
 feature1      0.000862
 feature2      0.001256
 feature3     -0.019092
 feature4      0.005373
                 ...   
 feature295    0.005506
 feature296    0.001046
 feature297    0.022878
 feature298   -0.008804
 feature299    0.005385
 Length: 300, dtype: float64,
 feature0      0.036766
 feature1      0.036262
 feature2      0.029425
 feature3      0.044675
 feature4      0.034049
                 ...   
 feature295    0.045369
 feature296    0.038020
 feature297    0.072503
 feature298    0.047352
 feature299    0.043680
 Length: 300, dtype: float64)

In [30]:
# df3 = pd.DataFrame(df_ip['designation_embeddings'].to_list(), columns=feat_cols)

In [31]:
principalComponents_breast = pca_breast.fit_transform(x)

In [32]:
principal_breast_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['principal component 1', 'principal component 2'])

In [33]:
principal_breast_Df

,principal component 1,principal component 2
0,0.342696,-0.136891
1,-0.111243,0.030840
2,-0.111243,0.030840
3,-0.111243,0.030840
4,-0.111243,0.030840
...,...,...
382103,-0.111243,0.030840
382104,-0.111243,0.030840
382105,-0.111243,0.030840
382106,-0.111243,0.030840


In [34]:
df_pca = pd.concat([train_cps,principal_breast_Df], axis = 1)

In [35]:
df_pca.head()

,_id,company_products_services,CPS_embeddings,feature0,feature1,feature2,feature3,feature4,feature5,feature6,...,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,principal component 1,principal component 2
0,5b073192599b311ccc11b101,"[Personalization, Sales Development, Sales Com...","[0.058168184, 0.043260016, 0.0478967, -0.06373...",0.058168,0.04326,0.047897,-0.063734,0.032842,0.059778,-0.059143,...,0.112488,0.018943,-0.026005,-0.050847,0.001988,0.070341,-0.10811,0.001688,0.342696,-0.136891
1,5b0737d1599b311ccc11c3f4,[e-learning Herramienta colaborativa],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-0.111243,0.030840
2,5b073905599b311ccc11c792,"[Power electronic systems, Advanced control ha...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-0.111243,0.030840
3,5b07552e599b311ccc121919,"[Education, Learning, Tutoring, Homework Help]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-0.111243,0.030840
4,5b0755f4599b311ccc121b43,[fine chemicals chemical sharing],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-0.111243,0.030840


In [36]:
df_pca = df_pca[["_id", "principal component 1", "principal component 2"]]

In [38]:
df_pca.head()

,_id,principal component 1,principal component 2
0,5b073192599b311ccc11b101,0.342696,-0.136891
1,5b0737d1599b311ccc11c3f4,-0.111243,0.030840
2,5b073905599b311ccc11c792,-0.111243,0.030840
3,5b07552e599b311ccc121919,-0.111243,0.030840
4,5b0755f4599b311ccc121b43,-0.111243,0.030840


In [39]:
len(df_pca)


# clustering find

382108

In [40]:
tqdm(df_pca.to_csv("../data/cps_embedding_pca.csv", index = False))

0it [00:00, ?it/s]

0it [00:00, ?it/s]